```
This notebook tests the projected files for the Oslofjord
Copyright (C) 2022 - 2023 SINTEF Digital
Copyright (C) 2022 - 2023 Norwegian Meteorological Institute

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
import sys
gpuocean_path = [p[:-4] for p in sys.path if p.endswith("gpuocean/src")][0]
import git
repo = git.Repo(gpuocean_path)
print("GPUOcean code from:", repo.head.object.hexsha, "on branch", repo.active_branch.name)

# Oslofjord
Testing of Nils projected files

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime, copy
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

In [ ]:
from gpuocean.utils import IPythonMagic, NetCDFInitialization

In [ ]:
%cuda_context_handler gpu_ctx

Path to the test file

In [ ]:
source_url = "/sintef/data/OsloFjord/test_polstere_24h_0007.nc"

## Inspecting file structure and content

In [ ]:
import xarray as xr
ds = xr.open_dataset(source_url)
ds

In [ ]:
from netCDF4 import Dataset
nc = Dataset(source_url)

H_m = np.ma.array(nc["h"][1:-1,1:-1], mask=(1-nc["mask_rho"][1:-1,1:-1]))

fig = plt.figure(figsize=(12,10))
plt.imshow(H_m)
plt.suptitle("Bathymetry")
plt.tight_layout()

Animation utils

In [ ]:
from IPython.display import clear_output
from matplotlib import animation, rc
plt.rcParams["animation.html"] = "jshtml"
from mpl_toolkits.axes_grid1 import make_axes_locatable

from gpuocean.utils import PlotHelper
from gpuocean.utils.NetCDFInitialization import depth_integration

def plotSolution(fig, 
                 eta, hu, hv, h, dx, dy, 
                 t, red_grav_mode=False,
                 comment = "Oslo",
                 h_min=-0.25, h_max=0.25, 
                 uv_min=-5, uv_max=5,
                 ax=None, sp=None, quiv=None):


    from datetime import timedelta
    fig.suptitle("Time = " + str(datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')) + " " + comment, 
                 fontsize=18,
                 horizontalalignment='left')
    
    ny, nx = eta.shape
    domain_extent = [0, nx*dx, 0, ny*dy]
    
    x_plots = 4
    y_plots = 1
   
    huv_label = ["hv","hu"]

    # Prepare quiver
    u = hu/(h+eta)
    v = hv/(h+eta)
    velocity = np.sqrt(u*u + v*v)
    
    frequency_x = 10
    frequency_y = 10
    x = np.arange(0, velocity.shape[1], frequency_x)*dx
    y = np.arange(0, velocity.shape[0], frequency_y)*dy
    qu = u[::frequency_y, ::frequency_x]
    qv = v[::frequency_y, ::frequency_x]

    if red_grav_mode:
        eta = -(h+eta)
        h_min = -15
        h_max = 0

    if (ax is None):
        ax = [None]*x_plots*y_plots
        sp = [None]*x_plots*y_plots

        uv_cmap = plt.cm.coolwarm
        uv_cmap.set_bad("grey", alpha = 1.0)
        
        h_cmap = plt.cm.coolwarm
        h_cmap.set_bad("grey", alpha = 1.0)
        if red_grav_mode:
            h_cmap = plt.cm.Blues_r
            h_cmap.set_bad("grey", alpha = 1.0)

        velo_cmap = plt.cm.Reds
        velo_cmap.set_bad("grey", alpha = 1.0)

        ax[0] = plt.subplot(y_plots, x_plots, 1)
        sp[0] = ax[0].imshow(eta, interpolation="none", origin='lower', 
                             cmap=h_cmap, 
                             vmin=h_min, vmax=h_max, 
                             extent=domain_extent)
        plt.axis('image')
        plt.title("MLD")
        divider0 = make_axes_locatable(ax[0])
        cax0 = divider0.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(sp[0],cax=cax0)


        ax[1] = plt.subplot(y_plots, x_plots, 2)
        sp[1] = ax[1].imshow(hu, interpolation="none", origin='lower', 
                            cmap=uv_cmap, 
                            vmin=uv_min, vmax=uv_max, 
                            extent=domain_extent)
        plt.axis('image')
        plt.title("$"+huv_label[0]+"$")
        divider1 = make_axes_locatable(ax[1])
        cax1 = divider1.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(sp[1],cax=cax1)



        ax[2] = plt.subplot(y_plots, x_plots, 3)
        sp[2] = ax[2].imshow(hv, interpolation="none", origin='lower', 
                             cmap=uv_cmap, 
                             vmin=uv_min, vmax=uv_max, 
                             extent=domain_extent)
        plt.axis('image')
        plt.title("$"+huv_label[1]+"$")
        divider2 = make_axes_locatable(ax[2])
        cax2 = divider2.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(sp[2],cax=cax2)

        ax[3] = plt.subplot(y_plots, x_plots, 4)
        sp[3] = ax[3].imshow(velocity, interpolation="none", origin='lower', 
                             cmap="Reds", 
                             vmin=0, vmax=1.0, 
                             extent=domain_extent)
        quiv = ax[3].quiver(x,y,qu,qv, scale=1)
        plt.axis('image')
        plt.title("velocity")
        divider2 = make_axes_locatable(ax[3])
        cax3 = divider2.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(sp[3],cax=cax3)

        plt.tight_layout()
            
    else:        
        #Update plots
        fig.sca(ax[0])
        sp[0].set_data(eta)

        fig.sca(ax[1])
        sp[1].set_data(hu)
        
        fig.sca(ax[2])
        sp[2].set_data(hv)

        fig.sca(ax[3])
        sp[3].set_data(velocity)
        quiv.set_UVC(qu, qv)
    
    return ax, sp, quiv


def ncAnimation(filename, nctype, ROMS_upper_layer=None, ROMS_upper_layer_coord=[0,None,0,None], comment="",**kwargs):
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(20, 6))

    ncfile = Dataset(filename)

    red_grav_mode = False

    if nctype == "ROMS":
        t = ncfile.variables['ocean_time'][:]

        H_m = np.ma.array(ncfile["h"][1:-1,1:-1], mask=[1-ncfile["mask_rho"][1:-1,1:-1]])

        if ROMS_upper_layer is None:
            eta = np.ma.array(ncfile["zeta"][:,1:-1,1:-1], mask=len(t)*[1-ncfile["mask_rho"][1:-1,1:-1]])
            try:
                u = np.ma.array( 0.5*(ncfile["ubar"][:,1:-1,1:]+ncfile["ubar"][:,1:-1,:-1]), mask=len(t)*[1-ncfile["mask_rho"][1:-1,1:-1]])
                v = np.ma.array( 0.5*(ncfile["vbar"][:,1:,1:-1]+ncfile["vbar"][:,:-1,1:-1]), mask=len(t)*[1-ncfile["mask_rho"][1:-1,1:-1]])

                hu = u*H_m
                hv = v*H_m
            except:
                u = 0.5*(ncfile["u"][:,:,1:-1,1:]+ncfile["u"][:,:,1:-1,:-1])
                v = 0.5*(ncfile["v"][:,:,1:,1:-1]+ncfile["v"][:,:,:-1,1:-1])
                
                integrator = NetCDFInitialization.MLD_integrator(source_url, H_m, x0=1, x1=-1, y0=1, y1=-1)
                hu = np.ma.array(np.sum(integrator * u, axis=1), mask=len(t)*[1-ncfile["mask_rho"][1:-1,1:-1]])
                hv = np.ma.array(np.sum(integrator * v, axis=1), mask=len(t)*[1-ncfile["mask_rho"][1:-1,1:-1]])
        else:
            x0, x1 = ROMS_upper_layer_coord[0], ROMS_upper_layer_coord[1]
            y0, y1 = ROMS_upper_layer_coord[2], ROMS_upper_layer_coord[3]
            u = 0.5*(ncfile["u"][:,:,y0:y1,x0:x1]+ncfile["u"][:,:,y0:y1,x0+1:x1+1])
            v = 0.5*(ncfile["v"][:,:,y0:y1,x0:x1]+ncfile["v"][:,:,y0+1:y1+1,x0:x1])
            
            integrator = NetCDFInitialization.MLD_integrator(source_url, ROMS_upper_layer, x0=x0, x1=x1, y0=y0, y1=y1)
            hu = np.ma.array(np.sum(integrator * u, axis=1), mask=len(t)*[1-ncfile["mask_rho"][y0:y1,x0:x1]])
            hv = np.ma.array(np.sum(integrator * v, axis=1), mask=len(t)*[1-ncfile["mask_rho"][y0:y1,x0:x1]])

            eta = np.ma.array(len(t)*[ROMS_upper_layer], mask=len(t)*[ROMS_upper_layer.mask])
            H_m = 0.0

            red_grav_mode = True


    elif nctype == "gpuocean":
        t = ncfile["time"][:]

        eta = ncfile["eta"][:]
        hu  = ncfile["hu"][:]
        hv  = ncfile["hv"][:]

        H_m = ncfile["Hm"][:]

    elif nctype == "gpuocean-reduced_grav":
        t = ncfile["time"][:]

        eta = ncfile["eta"]
        hu  = ncfile["hu"][:]
        hv  = ncfile["hv"][:]

        H_m = ncfile["Hm"][:]
        
        red_grav_mode = True

        

    movie_frames = len(t)

    dx = 50
    dy = 50
    
    ax, sp, quiv = plotSolution(fig, 
                            eta[0],
                            hu[0],
                            hv[0],
                            H_m,
                            dx, dy, 
                            t[0], 
                            red_grav_mode,
                            comment=comment,
                            **kwargs)


    #Helper function which simulates and plots the solution    
    def animate(i):
        t_now = t[0] + (i / (movie_frames-1)) * (t[-1] - t[0]) 

        k = np.searchsorted(t, t_now)
        if (k >= eta.shape[0]):
            k = eta.shape[0] - 1
        j = max(0, k-1)
        if (j == k):
            k += 1
        s = (t_now - t[j]) / (t[k] - t[j])

        plotSolution(fig, 
                        ((1-s)*eta[j] + s*eta[k]), 
                        ((1-s)*hu[j]  + s*hu[k]), 
                        ((1-s)*hv[j]  + s*hv[k]), 
                        (H_m+(1-s)*eta[j] + s*eta[k]), 
                        dx, dy, 
                        t_now, 
                        red_grav_mode,
                        comment=comment,
                        **kwargs, ax=ax, sp=sp, quiv=quiv)

        clear_output(wait = True)
        #print(progress.getPrintString(i / (movie_frames-1)))

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(movie_frames), interval=250)
    plt.close(fig)
    
    return anim


## Generating GPUOcean Simulation from Input

Initialisation (actually barotropic model values, but we gonna replace them bit by bit)

In [ ]:
dimY, dimX = ds.h.data.shape

In [ ]:
x0, x1, y0, y1 = 5, dimX-5, 135, dimY-5

In [ ]:
from importlib import reload
reload(NetCDFInitialization)

data_args = NetCDFInitialization.getInitialConditions(source_url, x0, x1, y0, y1, norkyst_data=False, download_data=False, land_value=0.0, t0_index=1)

In [ ]:
data_args["dx"], data_args["dy"]

Check Mixed-Layer Depth

In [ ]:
mld = NetCDFInitialization.MLD(source_url, 1025, min_mld=3, max_mld=40, x0=x0, x1=x1, y0=y0, y1=y1, t=1)

In [ ]:
s_nc = Dataset(source_url)
s_hs = s_nc["h"][y0:y1,x0:x1]

bad_yx = np.where(np.logical_or(np.abs(mld - s_hs) < 0.35*s_hs, np.abs(mld - s_hs) < 3))
bad_mask = np.where(np.logical_and((s_hs!=0.0), np.logical_and(np.abs(mld - s_hs) > 0.35*s_hs, np.abs(mld - s_hs) > 3)),0,1)

Xidx = np.arange(0, mld.shape[1])
Yidx = np.arange(0, mld.shape[0])

xx, yy = np.meshgrid(Xidx, Yidx)

K = 20

for i in range(len(bad_yx[0])):
    dists = (xx-bad_yx[1][i])**2 + (yy-bad_yx[0][i])**2 + 1e5*bad_mask
    sum = 0.0
    for k in range(K): 
        sum += mld[np.unravel_index(dists.argmin(), dists.shape)]
        dists[np.unravel_index(dists.argmin(), dists.shape)] = 1e5
    mld[bad_yx[0][i],bad_yx[1][i]] = sum/K

In [ ]:
mld = NetCDFInitialization.fill_coastal_data(mld)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,10))

im = axs[0].imshow(np.ma.array(ds.h.data[y0:y1,x0:x1], mask=(ds.h.data[y0:y1,x0:x1]==0)), cmap="cool")
plt.colorbar(im, ax=axs[0], shrink=0.5)
axs[0].set_title("Bathymetry")

im = axs[1].imshow(mld, cmap="cool", vmin=0.0, vmax=15)
plt.colorbar(im, ax=axs[1], shrink=0.5)
axs[1].set_title("MLD")

im = axs[2].imshow(bad_mask-(s_hs==0), cmap="Reds")
plt.colorbar(im, ax=axs[2], shrink=0.5)
axs[2].set_title("Bad values")



In [ ]:
y_cut = 120

s_pot_densities = NetCDFInitialization.potentialDensities(source_url, t=0, x0=x0, x1=x1, y0=y0, y1=y1)

s_nc = Dataset(source_url)
s_hs   = s_nc["h"][y0:y1,x0:x1]
s_rhos = s_nc["Cs_r"][:]

s_pot_densities_show = np.ma.array(np.zeros((s_pot_densities.shape[1],100)))
for l in range(s_pot_densities.shape[1]):
    d_up = 0 
    for i in reversed(range(len(s_rhos))):
        d = round(-(s_hs[l,y_cut]*s_rhos[i]))
        s_pot_densities_show[l,d_up:d] = s_pot_densities[i][l,y_cut]
        d_up = d
s_pot_densities_show.mask = (s_pot_densities_show==0)

plt.figure(figsize=(15,5))
plt.imshow(s_pot_densities_show.T[0:50], origin="upper", cmap="plasma",  aspect='auto')
plt.colorbar()
plt.scatter(np.arange(s_pot_densities.shape[1]), mld[:,y_cut])

In [ ]:
# TEMP!
mld = np.ma.array( 5*np.ones_like(mld), mask=copy.copy(mld.mask) )

In [ ]:
ncAnimation(source_url, "ROMS", ROMS_upper_layer=mld, ROMS_upper_layer_coord=[x0,x1,y0,y1], comment="Uppper layer of FjordOS")

#### Baroclinic model

In [ ]:
H = 3.0

In [ ]:
data_args["H"] = np.ma.array(H*np.ones_like(data_args["H"]), mask=data_args["H"].mask.copy(), dtype=np.float32)

In [ ]:
data_args["eta0"] = np.ma.array(mld.data - H, mask=copy.copy(mld.mask))

In [ ]:
ml_integrator = NetCDFInitialization.MLD_integrator(source_url, mld, x0=x0, x1=x1, y0=y0, y1=y1)

In [ ]:
t0_index = 5

nc = Dataset(source_url)
u0 = nc.variables['u'][t0_index, :, y0:y1, x0:x1+1]
v0 = nc.variables['v'][t0_index, :, y0:y1+1, x0:x1]
#Find u,v at cell centers
u0 = u0.filled(fill_value = 0.0)
v0 = v0.filled(fill_value = 0.0)

u0 = (u0[:, :,1:] + u0[:, :, :-1]) * 0.5
v0 = (v0[:, 1:,:] + v0[:, :-1, :]) * 0.5

data_args["hu0"] = np.sum(ml_integrator * u0, axis=0)
data_args["hv0"] = np.sum(ml_integrator * v0, axis=0)

In [ ]:
# # Starting from lake at rest
# data_args["hu0"] = np.ma.array(np.zeros_like(mld), mask=copy.copy(mld.mask))
# data_args["hv0"] = np.ma.array(np.zeros_like(mld), mask=copy.copy(mld.mask))

In [ ]:
s_pot_densities = NetCDFInitialization.potentialDensities(source_url, t=0, x0=x0, x1=x1, y0=y0, y1=y1)
ml_pot_density = np.average(np.sum(ml_integrator * s_pot_densities, axis=0)/np.sum(ml_integrator, axis=0)) #NOTE: np.sum(integrator, axis=0)) = mld

inverse_integrator = np.ma.array(np.ones_like(ml_integrator), mask=ml_integrator.mask.copy()) - ml_integrator
deep_pot_density  = np.average(np.sum(inverse_integrator * s_pot_densities, axis=0)/np.sum(inverse_integrator, axis=0))

eps = (deep_pot_density - ml_pot_density)/deep_pot_density

data_args["g"] = eps*data_args["g"] 
data_args["g"]

In [ ]:
# data_args["g"] = 0.01

Set-up osciallating BC

In [ ]:
from gpuocean.utils import Common

In [ ]:
NX = data_args["nx"]+4
NY = data_args["ny"]+4

In [ ]:
freq = 12*3600 #Input
T = 3*24*3600  #Input

t_step = freq/12
T_steps = int(np.ceil(T/t_step)+1)

ts = np.arange(0, T+1, step=t_step)

In [ ]:
bc_h = np.tile(mld[0], (T_steps,1))

In [ ]:
ampl = 0.5 #Input
bc_v = ampl*np.ones((T_steps, NX)) * np.sin(2*np.pi*ts/freq)[:,np.newaxis] + ampl/2

In [ ]:
bc_hv = bc_h*bc_v

bc_h = bc_h - H

In [ ]:
south = Common.SingleBoundaryConditionData(h=bc_h.astype(np.float32), hu=np.zeros((T_steps, NX), dtype=np.float32), hv=bc_hv.astype(np.float32))
north = Common.SingleBoundaryConditionData(h=np.zeros((T_steps, NX), dtype=np.float32), hu=np.zeros((T_steps, NX), dtype=np.float32), hv=np.zeros((T_steps, NX), dtype=np.float32))
east  = Common.SingleBoundaryConditionData(h=np.zeros((T_steps, NY), dtype=np.float32), hu=np.zeros((T_steps, NY), dtype=np.float32), hv=np.zeros((T_steps, NY), dtype=np.float32))
west  = Common.SingleBoundaryConditionData(h=np.zeros((T_steps, NY), dtype=np.float32), hu=np.zeros((T_steps, NY), dtype=np.float32), hv=np.zeros((T_steps, NY), dtype=np.float32))

In [ ]:
data_args["boundary_conditions_data"] = Common.BoundaryConditionsData(ts, north=north, south=south, east=east, west=west)

In [ ]:
plt.plot((ts/3600)[:int(24*3600/t_step)], bc_v[:int(24*3600/t_step),0])
plt.title("current forcing from the south boundary")
plt.xlabel("time [h]")
plt.ylabel("u [m^2/s]")

In [ ]:
from gpuocean.utils import WindStress
data_args["wind_stress"] = WindStress.WindStress(t=[0],X=[np.array([[0.0000]], dtype=np.float32)],Y=[np.array([[0.000]], dtype=np.float32)])

In [ ]:
from gpuocean.SWEsimulators import CDKLM16
osc_sim = CDKLM16.CDKLM16(gpu_ctx, dt=0.0,  **NetCDFInitialization.removeMetadata(data_args), write_netcdf=True)

In [ ]:
for t in range(3*24):#range(len(ds.ocean_time)):
    osc_sim.step(3600)

In [ ]:
ncAnimation(osc_sim.sim_writer.output_file_name, "gpuocean-reduced_grav")

In [ ]:
osc_sim.boundary_conditions.spongeCells